In [2]:
import pandas as pd
import numpy as np

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning.metrics import F1
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers import TensorBoardLogger

In [4]:
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    classification_report,
    confusion_matrix,
)

In [5]:
import pickle

In [6]:
with open("../data/1_interim/X_and_y.pkl","rb") as f:
    x, y = pickle.load(f)

In [7]:
x

,BTC,ETH,LTC
2018-06-01,8.921407,6.347154,4.766226
2018-06-02,8.925100,6.358288,4.786947
2018-06-03,8.939896,6.382461,4.813993
2018-06-04,8.951070,6.426978,4.831233
2018-06-05,8.923403,6.382745,4.783500
...,...,...,...
2021-04-17,11.034338,7.799110,5.741568
2021-04-18,11.011628,7.772650,5.742972
2021-04-19,10.948186,7.718245,5.623626
2021-04-20,10.937101,7.696962,5.602198


In [8]:
y

,BTC,ETH,LTC
2018-07-21,-1,-1,-1
2018-07-22,1,1,1
2018-07-23,0,-1,-1
2018-07-24,1,-1,0
2018-07-25,1,1,1
...,...,...,...
2021-04-17,-1,-1,1
2021-04-18,-1,-1,0
2021-04-19,-1,-1,-1
2021-04-20,-1,-1,-1


In [22]:
time_series = x.BTC.to_numpy()
label = y.BTC.apply(lambda y: 2 if y == -1 else y).to_numpy()

In [47]:
N_CLASSES = 3 ###
N_CURRENCIES = 1
INPUT_FEATURE_SIZE = 1
WINDOW_SIZE = 50
TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = 0.80, 0.10, 0.10

In [48]:
#train dataset ile val dataset ayırabilirim okunması daha kolay olsun diye aslında

In [59]:
class TimeSeriesDataset(Dataset):
    def __init__(self, 
                 x: np.ndarray, 
                 y: np.ndarray,
                 data_use_type,
                 train_percentage = TRAIN_PERCENTAGE,
                 val_percentage = VAL_PERCENTAGE,
                 test_percentage = TEST_PERCENTAGE,
                 seq_len = WINDOW_SIZE, 
                 ):
        
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).long()
        self.seq_len = seq_len
        
        self.data_use_type = data_use_type
        
        self.train_size = int(len(self.x) * train_percentage)
        self.val_size = int(len(self.x) * val_percentage)
        self.test_size = int(len(self.x) * test_percentage)
        
        self.train_mean = self.x[:self.train_size].mean()
        self.train_std = self.x[:self.train_size].std()
        
        self.train_min = self.x[:self.train_size].min()
        self.train_max = self.x[:self.train_size].max()
        
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - ( self.seq_len)
        
        if self.data_use_type == "val":
            return self.val_size
        
        else:
            return self.test_size
        
    def __getitem__(self, index):
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        window = self.x[index:index+self.seq_len]
        window = (window -self.train_mean) / self.train_std 
        
        price_change = self.y[index+self.seq_len]
        #burada 2 hiç gelmiyor N_CLASSES = 2 dediğinde run edince patlardı
        return (window, price_change)

In [60]:
train_dataset = TimeSeriesDataset(time_series, label,"train")
val_dataset = TimeSeriesDataset(time_series, label, "val")
test_dataset = TimeSeriesDataset(time_series, label, "test")

In [61]:
test_dataset.__len__()

105

In [62]:
#calculate loss' weights
train_labels = [train_dataset[i][1] for i in range (train_dataset.__len__())]
samples_size = pd.DataFrame({"label": train_labels}).groupby("label").size().to_numpy()
loss_weights = (1 / samples_size) * sum(samples_size)/2
loss_weights

array([132.33333333,  99.25      , 397.        , 198.5       ,
       132.33333333, 397.        ,  99.25      ,  99.25      ,
       132.33333333, 198.5       , 198.5       ,  66.16666667,
       198.5       , 132.33333333, 397.        ,  99.25      ,
       397.        , 397.        , 397.        ,  99.25      ,
       198.5       , 397.        , 198.5       , 198.5       ,
       132.33333333, 198.5       , 397.        , 397.        ,
       397.        , 198.5       , 198.5       , 198.5       ,
       132.33333333, 198.5       ,  99.25      , 198.5       ,
        99.25      , 397.        , 132.33333333, 397.        ,
       198.5       , 397.        , 132.33333333, 397.        ,
       397.        ,  79.4       , 397.        , 132.33333333,
       132.33333333, 198.5       ,  28.35714286,  56.71428571,
       397.        ,  18.9047619 , 198.5       ,  30.53846154,
        99.25      ,  24.8125    , 132.33333333,  36.09090909,
       198.5       , 397.        , 132.33333333, 132.33

In [63]:
train_labels

[tensor(2),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(2),
 tensor(0),
 tensor(1),
 tensor(2),
 tensor(1),
 tensor(1),
 tensor(2),
 tensor(1),
 tensor(1),
 tensor(2),
 tensor(2),
 tensor(2),
 tensor(1),
 tensor(1),
 tensor(2),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(2),
 tensor(2),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(2),
 tensor(2),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(2),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(2),
 tensor(1),
 tensor(2),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(2),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(2),
 tensor(2),
 tensor(1),
 tensor(2),
 tensor(0),
 tensor(2),
 tensor(2),
 tensor(2),
 tensor(2),
 tensor(1),
 tensor(1),
 tensor(2),
 tensor(2),
 tensor(1),
 tensor(2),
 tensor(2),
 tensor(2),
 tensor(1),
 tensor(2),
 tensor(1),
 tensor(1),
 tensor(1),
 ten

In [1]:
class LSTM_based_classification_model(pl.LightningModule):
    def __init__(self,
                 train_dataset = train_dataset,
                 val_dataset = val_dataset,
                 test_dataset = test_dataset,
                 weights = loss_weights,
                 num_classes = N_CLASSES,
                 window_size = WINDOW_SIZE,
                 input_size = INPUT_FEATURE_SIZE,
                 batch_size=8,
                 lstm_hidden_size = 256,
                 lstm_stack_size = 1,
                 lstm_dropout = 0.2,
                 bidirectional = False,
                 ):
        
        super().__init__()
        self.num_classes = num_classes
        self.window_size = window_size
        self.input_size = input_size
        self.batch_size = batch_size
        
        self.lstm_hidden_size = lstm_hidden_size
        self.lstm_stack_size = lstm_stack_size
        self.lstm_dropout = lstm_dropout
        self.bidirectional = bidirectional 
        
#         self.stack_lstm = nn.LSTM(input_size = self.input_size, 
#                 hidden_size = self.lstm_hidden_size, 
#                 num_layers= self.lstm_stack_size,
#                 dropout = self.lstm_dropout, # sadece stack arasına koyuyor
#                 bidirectional = self.bidirectional, 
#                 batch_first=True,) 
        
        self.lstm_1 = nn.LSTM(input_size = self.input_size, num_layers=1, batch_first=True, hidden_size = 256)
        self.batch_norm1 = nn.BatchNorm2d(num_features=256)
        
        self.lstm_2 = nn.LSTM(input_size = 256, num_layers=1, batch_first=True, hidden_size = 128)
        self.batch_norm2 = nn.BatchNorm2d(num_features=128)
        
        self.lstm_3 = nn.LSTM(input_size = 128, num_layers=1, batch_first=True, hidden_size = 64)
        self.batch_norm3 = nn.BatchNorm2d(num_features=64)
        
        self.dropout = nn.Dropout(0.5)
        
        #         self.linear1 = nn.Linear(self.lstm_hidden_size, 128)
        
        self.linear1 = nn.Linear(64, 128)
        
        self.activation = nn.ReLU()
        
        
        
        self.output_layer = nn.Linear(128, self.num_classes)
        
        #self.cross_entropy_loss = nn.CrossEntropyLoss(weight= torch.tensor(weights).float()) # loss weight
        self.cross_entropy_loss = nn.CrossEntropyLoss()
        
        self.f1_score = pl.metrics.F1(num_classes=self.num_classes, average="macro")
        self.accuracy_score = pl.metrics.Accuracy()
        
        self.train_dl = DataLoader(train_dataset, batch_size=self.batch_size)
        self.val_dl = DataLoader(val_dataset, batch_size=self.batch_size)
        self.test_dl = DataLoader(test_dataset, batch_size=self.batch_size)
        
    def forward(self, x):
        batch_size = x.size()[0]
        x = x.view(batch_size, self.window_size, self.input_size) #(batch, window_len, feature_size)
        x, _  = self.lstm_1(x)
        x = self.dropout(x)
        
        x = x.reshape(256, batch_size, self.window_size) #(batch, window_len, feature_size)
        x = self.batch_norm1(x.unsqueeze(0))
        
        x = x.view(batch_size, self.window_size, 256)
        x, _  = self.lstm_2(x)
        x = self.dropout(x)
        
        x = x.reshape(128, batch_size, self.window_size) #(batch, window_len, feature_size)
        x = self.batch_norm2(x.unsqueeze(0))
        #         x, _ = self.lstm2(x)
        
        x = x.view(batch_size, self.window_size, 128)
        x, _  = self.lstm_3(x)
        x = self.dropout(x)
        
        x = x.reshape(64, batch_size, self.window_size) #(batch, window_len, feature_size)
        x = self.batch_norm3(x.unsqueeze(0))
        
        x = x.view(batch_size, self.window_size, 64)
        x = x[:, -1, :] # equivalent to return sequence = False on keras :)
        
#         x = self.dropout(x)
        
        x = self.linear1(x)
        x = self.activation(x)
        x = self.dropout(x)
        
#         x = self.linear2(x)
#         x = self.activation(x)
#         x = self.dropout(x)
            
        output = self.output_layer(x)
        
        #output = F.log_softmax(output, dim = 1)
        #output = F.softmax(output)
        
        return output
    
    def training_step(self, batch, batch_nb):
        x, y = batch
        output = self(x)
        #loss = F.nll_loss(output, y)
        loss = self.cross_entropy_loss(output, y)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        
        acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
        self.log('train_acc', acc, on_epoch=True, prog_bar=True)
        
        f1 = self.f1_score(torch.max(output, dim=1)[1], y)
        self.log('train_f1', f1, on_epoch=True, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_nb):
        x, y = batch
        output = self(x)
        #loss = F.nll_loss(output, y)
        loss = self.cross_entropy_loss(output, y)
        self.log('val_loss', loss, on_epoch=True, reduce_fx=torch.mean, prog_bar=True)
        
        #print(torch.max(output, dim=1)[1])
        acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
        self.log('val_acc', acc, on_epoch=True, reduce_fx=torch.mean, prog_bar=True)
        
        f1 = self.f1_score(torch.max(output, dim=1)[1], y)
        self.log('val_f1', f1, on_epoch=True, reduce_fx=torch.mean, prog_bar=True)
        
    def test_step(self, batch, batch_nb):
        x, y = batch
        output = self(x)
        
        #print(y, torch.max(output, dim=1)[1])
        #print(output)
        print(F.softmax(output)) # mantıken fark etmiyor
        print(y, torch.max(output, dim=1)[1])
        
        #loss = F.nll_loss(output, y)
        loss = self.cross_entropy_loss(output, y)
        self.log('test_loss', loss, on_epoch=True, reduce_fx=torch.mean)
        
        acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
        self.log('test_acc', acc, on_epoch=True, reduce_fx=torch.mean)
        
        f1 = self.f1_score(torch.max(output, dim=1)[1], y)
        self.log('test_f1', f1, on_epoch=True, reduce_fx=torch.mean)
        
    def configure_optimizers(self):
        
        optimizer = torch.optim.AdamW(model.parameters(), lr= 1e-3)#AdamW does weight decay
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

        return [optimizer], [{"scheduler": scheduler}]
    
    def train_dataloader(self):
        return self.train_dl

    def val_dataloader(self):
        return self.val_dl

    def test_dataloader(self):
        return self.test_dl

NameError: name 'pl' is not defined

In [2]:
!rm -rf ../output/models/lstm_v1/version_*

In [66]:
#logger = WandbLogger(name='lstm.v4',project='pytorchlightning')
logger = TensorBoardLogger("../output/models/lstm_model_logs", name="lstm_v1")

In [67]:
model = LSTM_based_classification_model(batch_size=8)
trainer = pl.Trainer(gpus=-1, 
                     logger = logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [68]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type             | Params
---------------------------------------------------------
0  | lstm_1             | LSTM             | 265 K 
1  | batch_norm1        | BatchNorm2d      | 512   
2  | lstm_2             | LSTM             | 197 K 
3  | batch_norm2        | BatchNorm2d      | 256   
4  | lstm_3             | LSTM             | 49.7 K
5  | batch_norm3        | BatchNorm2d      | 128   
6  | dropout            | Dropout          | 0     
7  | linear1            | Linear           | 8.3 K 
8  | activation         | ReLU             | 0     
9  | output_layer       | Linear           | 387   
10 | cross_entropy_loss | CrossEntropyLoss | 0     
11 | f1_score           | F1               | 0     
12 | accuracy_score     | Accuracy         | 0     
---------------------------------------------------------
522 K     Trainable params
0         Non-trainable params
522 K     Total params
2.088     Total estimated model 

/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [69]:
train_dataset[0][1]

tensor(2)

In [70]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


<ipython-input-64-2594ed4ba110>:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(output)) # mantıken fark etmiyor


tensor([[0.2918, 0.3979, 0.3103],
        [0.2902, 0.4002, 0.3095],
        [0.2914, 0.3988, 0.3098],
        [0.2912, 0.4001, 0.3087],
        [0.2921, 0.3994, 0.3085],
        [0.2919, 0.3992, 0.3089],
        [0.2913, 0.3998, 0.3089],
        [0.2933, 0.3966, 0.3101]], device='cuda:0')
tensor([2, 0, 0, 2, 1, 2, 1, 2], device='cuda:0') tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([[0.2938, 0.3965, 0.3097],
        [0.2916, 0.3995, 0.3089],
        [0.2925, 0.3982, 0.3093],
        [0.2921, 0.3996, 0.3084],
        [0.2927, 0.3993, 0.3081],
        [0.2923, 0.3993, 0.3084],
        [0.2913, 0.4002, 0.3084],
        [0.2932, 0.3971, 0.3097]], device='cuda:0')
tensor([1, 2, 1, 0, 1, 1, 1, 1], device='cuda:0') tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([[0.2937, 0.3968, 0.3095],
        [0.2916, 0.3996, 0.3088],
        [0.2923, 0.3987, 0.3090],
        [0.2917, 0.4002, 0.3081],
        [0.2923, 0.3999, 0.3079],
        [0.2918, 0.4000, 0.3082],
        [0.2906,

IndexError: index 1006 is out of bounds for dimension 0 with size 1006

In [151]:
test = TimeSeriesDataset(time_series, "test")
labels = [test[i][1] for i in range(test.__len__())]

In [152]:
pd.DataFrame({"label": labels}).groupby("label").size()

label
0    66
1    91
dtype: int64

In [ ]:
#dropout, batch normalization 

In [ ]:
183 / (183+131)